In [ ]:
import plotly.express as px
import numpy as np

depth_igev_data_cam = np.load('/home/mfi/repos/rtc_vision_toolbox/camera/zed_ros/tests/depth_igev_data.npy')
depth_igev_prob_cam = np.load('/home/mfi/repos/rtc_vision_toolbox/camera/zed_ros/tests/points_cam2_igev_0607_1311.ply')

# igev_cam = np.where((depth_igev_data_cam < 2000), depth_igev_data_cam, None)
# igev_prob = np.where((depth_igev_prob_cam > 0.5), depth_igev_prob_cam, None)
igev_cam = depth_igev_data_cam
igev_prob = depth_igev_prob_cam

max_val = np.nanmax(depth_igev_prob_cam)
min_val = np.nanmin(depth_igev_prob_cam)
print("max_val", max_val)
print("min_val", min_val)


fig = px.imshow(igev_prob)
fig.show()
fig = px.imshow(igev_cam)
fig.show()

In [16]:
import plotly.graph_objects as go

def plot_multi_np(plist):
    """
    Args: plist, list of numpy arrays of shape, (1,num_points,3)
    """
    colors = [
        '#1f77b4',  # muted blue
        '#ff7f0e',  # safety orange
        '#2ca02c',  # cooked asparagus green
        '#d62728',  # brick red
        '#9467bd',  # muted purple
    ]
    skip = 1
    go_data = []
    for i in range(len(plist)):
        p_dp = plist[i]
        plot = go.Scatter3d(x=p_dp[::skip,0], y=p_dp[::skip,1], z=p_dp[::skip,2], 
                     mode='markers', marker=dict(size=2, color=colors[i],
                     symbol='circle'))
        go_data.append(plot)
 
    layout = go.Layout(
        scene=dict(
            aspectmode='data',
        ),
        height=1200,
        width=1200,
    )
    fig = go.Figure(data=go_data, layout=layout)
    
    colors = ['red', 'green', 'blue']  # X, Y, Z axis colors
    
    fig = go.Figure()
    
    T = []
    T.append(np.eye(4))

    for tf in T:
        origin = tf[:3, 3]
        axes = tf[:3, :3]

        for i in range(3):
            axis_end = origin + 0.3*axes[:, i]
            fig.add_trace(go.Scatter3d(
                x=[origin[0], axis_end[0]],
                y=[origin[1], axis_end[1]],
                z=[origin[2], axis_end[2]],
                mode='lines',
                line=dict(color=colors[i], width=4),
                name='Axis ' + str(i+1)
            ))

    for plot in go_data:
        fig.add_trace(plot)
    
    # add axis lines and camera view
    fig.update_layout(scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
        camera = dict(
                      eye=dict(x=-1.30, y=0, z=-.25),
                      center=dict(x=0., y=0, z=-0.25),
                      up=dict(x=0, y=0, z=1),
                     )
        ),
        height=800,
        width=1200,
    )    
    
    fig.show()
    return fig

In [ ]:
T = np.array([
    [0, 0, 1, 0],
    [-1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, 0, 1]
])    

T_zed = np.linalg.inv(T)

In [17]:
import open3d as o3d
import copy
import numpy as np

pcd_file = '/home/mfi/repos/rtc_vision_toolbox/camera/zed_ros/tests/points_cam2_igev_0605_1746.ply'
tf_file = '/home/mfi/repos/rtc_vision_toolbox/data/calibration_data/zed/T_base2cam2.npy'

pcd_raw = o3d.io.read_point_cloud(pcd_file)
points_raw = np.asarray(pcd_raw.points) / 1000
points_raw[np.isinf(points_raw)] = np.nan
points_raw = points_raw[~np.isnan(points_raw).any(axis=1)]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_raw)

pcd = pcd.transform(T_zed) # only if using zed default point cloud

pcd_tf = copy.deepcopy(pcd)
T = np.load(tf_file)
transform = True
crop = True
downsample = False

if transform:
    pcd_tf.transform(T)

print("points.shape", np.asarray(pcd_tf.points).shape)

if crop:
    points = np.asarray(pcd_tf.points)
    # gripper boundaries
    gripper_boundaries = {
        'min': np.array([0.2, -0.3, 0.3]),
        'max': np.array([0.35, -0.10, .5])
    }
    world_boundaries = gripper_boundaries
    points[np.isinf(points)] = np.nan
    points[np.where(points[:, 0] < world_boundaries['min'][0])] = np.nan
    points[np.where(points[:, 1] < world_boundaries['min'][1])] = np.nan
    points[np.where(points[:, 2] < world_boundaries['min'][2])] = np.nan
    points[np.where(points[:, 0] > world_boundaries['max'][0])] = np.nan
    points[np.where(points[:, 1] > world_boundaries['max'][1])] = np.nan
    points[np.where(points[:, 2] > world_boundaries['max'][2])] = np.nan
    points = points[~np.isnan(points).any(axis=1)]
    pcd_tf = o3d.geometry.PointCloud()
    pcd_tf.points = o3d.utility.Vector3dVector(points)
    del points

print("points.shape", np.asarray(pcd_tf.points).shape)


if downsample:
    pcd_tf = pcd_tf.voxel_down_sample(voxel_size=0.05)
points = np.asarray(pcd_tf.points)
print("points.shape", points.shape)

plot_multi_np([points])

points.shape (719346, 3)
points.shape (7301, 3)
points.shape (7301, 3)
